# Full forward pass & decode with Mobilenet + ImageNet

## 1.- Imports

In [ ]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf
from matplotlib import pyplot as plt
from generate_mobilenet import Mobilenet
import pathlib
from tensorflow.python.keras import backend
from tensorflow.python.keras.utils import all_utils as keras_utils
import json
from mpl_toolkits.axes_grid1 import AxesGrid

In [ ]:
def plot_subplots(**kwargs):
    '''
    Function to plot several imagenes easily.
    kwargs:
    - dims: dimension of the subplots
    - imgs: list of images to plot
    '''
    with plt.style.context("dark_background"):
        fig = plt.figure(figsize=(6, 4))
        grid = AxesGrid(fig, 111,
                        nrows_ncols=kwargs['dims'],
                        axes_pad=0.05,
                        cbar_mode='single',
                        cbar_location='right',
                        cbar_pad=0.1
                        )
        for i, ax in enumerate(grid):
            ax.set_axis_off()
            im = ax.imshow(kwargs['imgs'][i], vmin=0, vmax=1)
        cbar = ax.cax.colorbar(im)
        cbar = grid.cbar_axes[0].colorbar(im)
        cbar.ax.set_yticks(np.arange(0, 1.1, 0.5))
        cbar.ax.set_yticklabels(['low', 'medium', 'high'])
        plt.show()
        plt.close(fig)

## 2.- Model instantiation

In [ ]:
path = pathlib.Path('.').parent.joinpath('debug', 'notebooks').resolve()
mobilenet = Mobilenet('MobileNet', path)

## 3.- ImageNet labels

In [ ]:
def get_imagenet_labels(last, num_classes=100):
    labels_path = tf.keras.utils.get_file(
        'ImageNetLabels.txt',
        'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
    imagenet_labels = np.array(open(labels_path).read().splitlines())
    # print('total # of labels: ', len(imagenet_labels))
    first_100 = imagenet_labels[-num_classes:]
    last_100 = imagenet_labels[:num_classes]
    imagenet_labels = last_100 if last else first_100
    # print('After prunning: ', imagenet_labels.shape)
    #print(f'''
    #Labels in partition:
    #\t- pizza: {'pizza' in imagenet_labels}
    #\t- scuba diver: {'scuba diver' in imagenet_labels}
    #\t- military uniform: {'military uniform' in imagenet_labels}
    #\t- dog: {'dog' in imagenet_labels}
    #\t- goldfish: {'goldfish' in imagenet_labels}
    #\t- salamander: {'spotted salamander' in imagenet_labels}
    #''')
    #print(imagenet_labels)
    return imagenet_labels

k = get_imagenet_labels(True)
k[2]

## 4.- Example image and preprocess
The two links provided are from TensorFlow and Wikipedia
![TensorFlow][https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg]
![Wikipedia][https://upload.wikimedia.org/wikipedia/commons/thumb/9/97/Dog_for_Senior_Dog_Food_Diet_Wikipedia_Page.jpg/1200px-Dog_for_Senior_Dog_Food_Diet_Wikipedia_Page.jpg]

In [ ]:
def preprocess_url(url, name):
    f = tf.keras.utils.get_file(name, url)
    img = tf.keras.preprocessing.image.load_img(f, target_size=[159, 159])
    img = tf.keras.preprocessing.image.img_to_array(img)
    x = tf.keras.applications.mobilenet.preprocess_input(img)
    x = (x-np.min(x))/(np.max(x)-np.min(x))
    return x[tf.newaxis,...]

def download_example_imgs():
    goldfish_url = 'https://i.ytimg.com/vi/DSDUCxvb7R4/hqdefault.jpg'
    pizza_url = 'https://upload.wikimedia.org/wikipedia/commons/2/2d/Punch_Neapolitan_Pizza_Margherita.jpg'
    scuba_diver_url = 'https://fthmb.tqn.com/VHeLk32zZhv3Kp70RBinGvF_dmM=/3920x2160/filters:fill(auto,1)/GettyImages-597572085-584819855f9b5851e593fd0b.jpg'
    hopper_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg"
    salamander1_url = 'https://upload.wikimedia.org/wikipedia/commons/6/6a/Blue-spotted_salamander_(Ambystoma_laterale).jpg'
    dog_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/9/97/Dog_for_Senior_Dog_Food_Diet_Wikipedia_Page.jpg/1200px-Dog_for_Senior_Dog_Food_Diet_Wikipedia_Page.jpg'

    goldfish = preprocess_url(goldfish_url, 'fish')
    pizza = preprocess_url(pizza_url, 'pizza')
    scuba = preprocess_url(scuba_diver_url, 'scuba')
    hopper = preprocess_url(hopper_url, 'hopper')
    salamander = preprocess_url(salamander1_url, 'salamand')
    dog = preprocess_url(dog_url, 'dog')
    imgs = [goldfish, pizza, scuba, hopper, salamander, dog]
    return imgs

In [ ]:
imgs = download_example_imgs()
print('Range of values: ', np.max(imgs[0]), np.min(imgs[0]))
plot_subplots(imgs=[im[0] for im in imgs], dims=(2,3))

# 5. Different classifier sizes and num_classes
The next cell will iterate over the number of classes of imagenet and it will create:
- a subset of labels for each case (from 800 clases to 10)
- a model which classifier has been loaded with that exact number weights for those classes on each iteration

It will perform a forward pass with an image of a goldfish and it will label the image correctly all the iterations.

The index of the 'goldfish' label is 2 so, it's safe to iterate from 1000 clases to 10 and still get the right output.


In [ ]:
for i in range(800, 2, -10):
    print(f"\n\n+====================================+")
    print(f"+==========={i}_NUM_CLASES===========+")
    print(f"+====================================+")
    mobilenet.build(num_classes=i)
    imagenet_labels = get_imagenet_labels(True, num_classes=i)
    output_goldfish = mobilenet.core_model(imgs[0])
    decoded = imagenet_labels[np.argsort(output_goldfish)[0,::-1][:1]+1]
    print(f"\n\n+====================================+")
    print(f"+=========DECODED_IMAGE OUTPUTS======+")
    print(f"+====================================+")
    print('(goldfish expected)\n>>',decoded)